In [ ]:
!pip install langchain_groq


In [ ]:
from langchain_groq import ChatGroq
llm= ChatGroq(
    temperature=0,
    groq_api_key ="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",
    model_name ="llama-3.3-70b-versatile"
)
result = llm.invoke("Who are you?")
print(result.content)

In [ ]:
!pip install pypdf


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader # Changed from PyPDFDirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq # Import ChatGroq
import os # Import os

def initialize_llm():
  llm = ChatGroq(
      temperature=0,
    groq_api_key ="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",
    model_name ="llama-3.3-70b-versatile"

  )
  return llm

def create_vector_db():
  # loader = DirectoryLoader("/content/data/", glob ='*.pdf', loader_cls = PyPDFDirectoryLoader) # Commented out DirectoryLoader
  loader = PyPDFLoader("/content/data/mental_health_Document_250625_161048.pdf") # Use PyPDFLoader with specific file path
  documents = loader.load()
  print(f"Number of documents loaded: {len(documents)}") # Added print statement
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceEmbeddings(model_name ='sentence-transformers/all-MiniLM-L6-v2') # Corrected typo
  vector_db = Chroma.from_documents(texts,embeddings, persist_directory='./chroma_db') # Corrected variable name
  vector_db.persist()

  print("ChromaDB created and data saved")
  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever=vector_db.as_retriever()
  prompt_template =""" You are a compassionate mental health chatbot.Respond thoughtfully to the following question:\n",
  "{context}\n",
  "User:{question}\n",
  " Chatbot:"""
  PROMPT =PromptTemplate(template = prompt_template, input_variables =['context','question'])
  qa_chain =RetrievalQA.from_chain_type(
      llm = llm,
      chain_type ="stuff",
      retriever = retriever ,
      chain_type_kwargs = {"prompt":PROMPT}

  )
  return qa_chain

def main():
  print("initializing Chatbot")
  llm = initialize_llm()

  db_path ="./chroma_db" # Corrected path
  if os.path.exists(db_path): # Added check for db_path existence
    print("Loading existing ChromaDB")
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2') # Corrected typo
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  else:
    print("Creating new ChromaDB")
    vector_db = create_vector_db()

  qa_chain= setup_qa_chain(vector_db,llm)

  while True:
    query =input("\nHuman:")
    if query.lower() == "exit":
      print("Chatbot:take care of yourself,Goodbye")
      break
    response = qa_chain.run(query)
    print(f"Chatbot:{response}")


if __name__ == "__main__":
  main()

In [ ]:
!pip install gradio


In [ ]:
create_vector_db()

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install langchain_groq langchain_core langchain_community

In [ ]:
!pip install Chromadb

In [ ]:
!mkdir data

In [ ]:
import os

print(os.listdir("/content/data/"))

In [ ]:
create_vector_db()
main()

In [ ]:
import os
print(os.listdir("/content/data/"))

In [ ]:
!mkdir data

In [ ]:
import os
print(os.listdir("/content/data/"))

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader # Changed from PyPDFDirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq # Import ChatGroq
import os # Import os
import gradio as gr

def initialize_llm():
  llm = ChatGroq(
      temperature=0,
    groq_api_key ="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",
    model_name ="llama-3.3-70b-versatile"

  )
  return llm

def create_vector_db():
  # loader = DirectoryLoader("/content/data/", glob ='*.pdf', loader_cls = PyPDFDirectoryLoader) # Commented out DirectoryLoader
  loader = PyPDFLoader("/content/data/mental_health_Document_250625_161048.pdf") # Use PyPDFLoader with specific file path
  documents = loader.load()
  print(f"Number of documents loaded: {len(documents)}") # Added print statement
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 50)
  texts = text_splitter.split_documents(documents)
  embeddings = HuggingFaceEmbeddings(model_name ='sentence-transformers/all-MiniLM-L6-v2') # Corrected typo
  vector_db = Chroma.from_documents(texts,embeddings, persist_directory='./chroma_db') # Corrected variable name
  vector_db.persist()

  print("ChromaDB created and data saved")
  return vector_db

def setup_qa_chain(vector_db, llm):
  retriever=vector_db.as_retriever()
  prompt_template =""" You are a compassionate mental health chatbot.Respond thoughtfully to the following question:\n",
  "{context}\n",
  "User:{question}\n",
  " Chatbot:"""
  PROMPT =PromptTemplate(template = prompt_template, input_variables =['context','question'])
  qa_chain =RetrievalQA.from_chain_type(
      llm = llm,
      chain_type ="stuff",
      retriever = retriever ,
      chain_type_kwargs = {"prompt":PROMPT}

  )
  return qa_chain

def chatbot_response(user_input, history): # Keep history parameter as Gradio passes it
  if not user_input.strip():
    return "please provide a valid input" # Return only the response string
  # Initialize LLM and QA chain within the response function or pass them
  llm = initialize_llm()
  db_path ="./chroma_db"
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  qa_chain= setup_qa_chain(vector_db,llm)

  response=qa_chain.run(user_input) # Call qa_chain.run with user_input
  return response # Return only the response string


if __name__ == "__main__":
  print("initializing Chatbot backend")
  llm = initialize_llm()

  db_path ="./chroma_db" # Corrected path
  if os.path.exists(db_path): # Added check for db_path existence
    print("Loading existing ChromaDB")
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2') # Corrected typo
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
  else:
    print("Creating new ChromaDB")
    vector_db = create_vector_db()

  qa_chain= setup_qa_chain(vector_db,llm)

  with gr.Blocks(theme= 'Repair/SHIKI@1.2.1')as app:
       gr.Markdown("Mental Health Chatbot ") # Fixed syntax error
       gr.Markdown("A compassionate chatbot designed to assist with mental well-being. Please note: For serious concerns, contact a professional.") # Fixed syntax error

       chatbot = gr.ChatInterface(fn=chatbot_response, title="Mental Health Chatbot")

       gr.Markdown("This chatbot provides general support. For urgent issues, seek help from licensed professionals.")

    # Modified fn to correctly pass history and expect a tuple return
    # Removed duplicate ChatInterface line
  app.launch(debug=True) # Added debug=True for more detailed output

In [ ]:
!pip install nbstripout
!nbstripout your_notebook.ipynb



In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
{'my_notebook.ipynb': 'my_notebook.ipynb'}


In [ ]:
!nbstripout my_notebook.ipynb


In [ ]:
from google.colab import files
uploaded = files.upload()
